# Part 2: "The Intern" (Fine-Tuning)

## Project 01 - Operation Ledger-Mind
**Course Module:** Weeks 01-03 (Prompt Engineering, Fine-Tuning, Advanced RAG)
**Scenario:** Financial Analysis of Uber Technologies (2024 Annual Report)

### 📋 Technical Requirements Checklist:
- [x] **Hugging Face Ecosystem**: transformers, peft, trl, bitsandbytes
- [x] **Base Model**: Llama-3-8b-Instruct
- [x] **Quantization**: 4-bit NF4 with double quantization
- [x] **Adapter Config**: LoRA (Targets: q_proj, k_proj, v_proj, o_proj)
- [x] **Training**: SFTTrainer for 100 steps
- [x] **Inference**: `query_intern(question)`

## 0. Environment Diagnostics

Verifying local hardware compatibility (RTX 3050 detected).

In [5]:
import torch
import sys
import os

print("="*60)
print("ENVIRONMENT & GPU CHECK")
print("="*60)
print(f"Python Version: {sys.version}")
print(f"PyTorch Version: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"Device Name: {torch.cuda.get_device_name(0)}")
    print(f"Memory Capability: {torch.cuda.get_device_capability(0)}")
    print(f"Total VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
    # RTX 3050 (Ampere) supports bfloat16
    print(f"BFloat16 Support: {torch.cuda.is_bf16_supported()}")
else:
    print("⚠️ WARNING: No CUDA GPU detected.")
    print("FIX: Run 'pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121'")
print("="*60)

ENVIRONMENT & GPU CHECK
Python Version: 3.12.2 (tags/v3.12.2:6abddd9, Feb  6 2024, 21:26:36) [MSC v.1937 64 bit (AMD64)]
PyTorch Version: 2.5.1+cu121
CUDA Available: True
Device Name: NVIDIA GeForce RTX 3050 Ti Laptop GPU
Memory Capability: (8, 6)
Total VRAM: 4.00 GB
BFloat16 Support: True


## 1. Setup & Configuration

Implementing 4-bit quantization and LoRA configuration per assessment specifications.

In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer
from datasets import load_dataset
from dotenv import load_dotenv

# Load Project Utils
sys.path.append(os.path.abspath("../"))
from src.services.llm_services import load_config

load_dotenv("../.env")
hf_token = os.getenv("HF_TOKEN")
config = load_config("../src/config/config.yaml")

base_model_id = config.get("base_model", "meta-llama/Meta-Llama-3-8B-Instruct")

# 4-bit Quantization Config (NF4, double quant)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16 if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else torch.float16
)

model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    token=hf_token
)

tokenizer = AutoTokenizer.from_pretrained(base_model_id, token=hf_token)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct.
403 Client Error. (Request ID: Root=1-698e9aaf-54f63a9e42ca16440e385da8;15722d45-4c69-425d-8d92-b63c1674642e)

Cannot access gated repo for url https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct/resolve/main/config.json.
Your request to access model meta-llama/Meta-Llama-3-8B-Instruct is awaiting a review from the repo authors.

## 2. Low-Rank Adaptation (LoRA)

Injecting trainable adapters into the attention modules.

In [ ]:
peft_config = LoraConfig(
    r=config.get("lora_r", 16),
    lora_alpha=config.get("lora_alpha", 32),
    lora_dropout=config.get("lora_dropout", 0.05),
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    bias="none",
    task_type="CAUSAL_LM"
)

model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

## 3. Data Formatting (ChatML Template)

Following the lecturer's pattern for instruction tuning on financial data.

In [ ]:
train_path = os.path.join("..", config['train_data_path'], 'train.jsonl')
dataset = load_dataset("json", data_files=train_path, split="train")

def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example['question'])):
        # Aligned with Teacher Workshop ChatML Structure
        text = f"<|im_start|>system\nYou are a professional financial analyst assistant. Answer questions based on Uber's 2024 Annual Report.<|im_end|>\n<|im_start|>user\n{example['question'][i]}<|im_end|>\n<|im_start|>assistant\n{example['answer'][i]}<|im_end|>"
        output_texts.append(text)
    return output_texts

## 4. Fine-Tuning Execution (SFTTrainer)

Training for the required 100 steps.

In [ ]:
training_args = TrainingArguments(
    output_dir="../artifacts/intern_checkpoints",
    per_device_train_batch_size=1, 
    gradient_accumulation_steps=64, 
    learning_rate=2e-5,
    logging_steps=10,
    max_steps=100, 
    save_steps=50,
    optim="paged_adamw_8bit",
    # Auto-select precision based on hardware
    fp16=True if torch.cuda.is_available() and not torch.cuda.is_bf16_supported() else False,
    bf16=True if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else False,
    report_to="none"
)

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    formatting_func=formatting_prompts_func,
    args=training_args,
    max_seq_length=1024
)

trainer.train()
trainer.save_model("../artifacts/intern_final_adapter")
print("✅ Training Complete. Adapters saved to artifacts/intern_final_adapter")

## 5. Inference Pipeline

Establishing `query_intern(question)` for downstream evaluation.

In [ ]:
def query_intern(question):
    prompt = f"<|im_start|>system\nYou are a professional financial analyst assistant. Answer questions based on Uber's 2024 Annual Report.<|im_end|>\n<|im_start|>user\n{question}<|im_end|>\n<|im_start|>assistant\n"
    
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda" if torch.cuda.is_available() else "cpu")
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs, 
            max_new_tokens=256, 
            temperature=0.1, 
            do_sample=True, 
            pad_token_id=tokenizer.eos_token_id
        )
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Extract response
    if "<|im_start|>assistant" in response:
        return response.split("<|im_start|>assistant")[-1].split("<|im_end|>")[0].strip()
    elif "assistant" in response:
        return response.split("assistant")[-1].strip()
    return response.strip()

# Sample baseline test
test_q = "What were the key drivers of Uber's revenue growth in 2024?"
print(f"Q: {test_q}")
if torch.cuda.is_available():
    print(f"A: {query_intern(test_q)}")
else:
    print("A: [CUDA not available, skipping inference for performance]")